<a href="https://colab.research.google.com/github/angeles-vinicius/AntonioAssist_ChatBot/blob/main/AntonioAssist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

"""
**AntonioAssist**🤖
- Chatbot para o Condomínio Antonio **Machado**

Este script implementa um chatbot chamado AntonioAssist, projetado para auxiliar moradores do meu condomínio, especialmente idosos,
a se conectarem com a comunidade e acessarem informações importantes.

Funcionalidades:

* Fornece informações sobre o condomínio: regras, horários, contatos, etc.
* Busca notícias relevantes (usando a API Gnews).
* Oferece previsão do tempo para qualquer cidade (usando a API OpenWeatherMap).
* Integração com um grupo de WhatsApp (implementação futura).

Objetivo:

Criar um ponto de auxílio e encontro virtual para os moradores, promovendo inclusão digital, interação social e bem-estar.

Baseado nos objetivos de desenvolvimento sustentável (ODS) da ONU ao qual reflete os objetivos desse chat para desenvolver o projeto.

Autor:
Angeles

Data:
Recife, 11 de maio de 2024.
"""


In [62]:
# Instala a biblioteca google-generativeai na versão mais recente e de forma silenciosa
!pip install -U -q google-generativeai
#Instala a implementação da capacidade de entender emoções dos moradores do codomínio através da API do Google Cloud Natural Language
!pip install --upgrade google-cloud-language

In [128]:
# Importações necessarias pro inicio do projeto

import numpy as np # Biblioteca para operações numéricas
import pandas as pd # Biblioteca para manipulação de dados
import google.generativeai as genai # Biblioteca para usar a API do Google Generative AI (Gemini)
import requests # Biblioteca para fazer requisições HTTP (para as APIs externas)
from bs4 import BeautifulSoup # Biblioteca para extrair dados de HTML (para a API de notícias, se necessário)
import re # Biblioteca para expressões regulares (para limpar o texto, se necessário)
from IPython.display import HTML, display # Para formatação da saída no Google Colab
from google.colab import userdata

In [129]:
# Melhora a visualização na saída, para incluir quebra de linha na visualização do chatbot.
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [130]:
# Necessario usar sua API Key real
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)
# API Keys para NewsAPI e OpenWeatherMap
NEWS_API_KEY = userdata.get('NEWS_API_KEY')
OPENWEATHERMAP_API_KEY = userdata.get('OPENWEATHERMAP_API_KEY')

In [131]:
# Necessário escolher os modelos Gemini para embedding e geração de texto.
model_embedding_str = "models/embedding-001"
model_generative_str = "gemini-1.0-pro"

In [132]:
# Configurações para geração de texto. Como o público alvo são os idoso, mensagens simples e clara se adapta melhor todos eles.
generation_config = {
  "temperature": 0.4,  # Controla a criatividade da resposta (0 = mais determinista, 1 = mais criativo)
  "candidate_count": 1 # Número de respostas candidatas a serem geradas
}

In [94]:
# Cria o modelo generativo
model_generative = genai.GenerativeModel(model_generative_str, generation_config=generation_config)

# Função para gerar embeddings de texto
def generate_embeddings(title, content):
  return genai.embed_content(
      model=model_embedding_str,
      content=content,
      title=title,
      task_type="RETRIEVAL_DOCUMENT"
  )["embedding"]

In [95]:
# Função para buscar a informação mais similar à pergunta do usuário
def search_similarity(query, df, model):
  query_embedding = genai.embed_content(
      model=model,
      content=query,
      task_type="RETRIEVAL_QUERY"
  )["embedding"]
  scalar_product = np.dot(np.stack(df["embedding"]), query_embedding)
  index = np.argmax(scalar_product)
  return df.iloc[index]["content"]

# Função para buscar notícias
def buscar_noticias(query=None):
  url = 'https://newsapi.org/v2/top-headlines?'
  if query:
    parameters = {'q': query, 'apiKey': NEWS_API_KEY, 'country': 'br', 'language': 'pt'}
  else:
    parameters = {'country': 'br', 'language': 'pt', 'apiKey': NEWS_API_KEY}
  response = requests.get(url, params=parameters)
  if response.status_code == 200:
    data = response.json()
    noticias = data['articles'][:3]
    return noticias
  else:
    return None

# Função para buscar previsão do tempo
def buscar_previsao_tempo(cidade):
  url = 'https://api.openweathermap.org/data/2.5/weather?'
  parameters = {'q': cidade, 'appid': OPENWEATHERMAP_API_KEY, 'units': 'metric', 'lang': 'pt'}
  response = requests.get(url, params=parameters)
  if response.status_code == 200:
    data = response.json()
    descricao = data['weather'][0]['description']
    temperatura = data['main']['temp']
    return f"A previsão do tempo para {cidade} é: {descricao}, com temperatura de {temperatura:.1f}°C."
  else:
    return None


In [96]:
# Crie um DataFrame com informações do condomínio
data = {
    "title": ["Regras do Condomínio", "Horário da área de lazer", "Portaria", "Contato do Síndico", "Abertura de chamados", "Acesso a plataforma de e-learning 60+"],
    "content": [
        "É proibido fazer barulho entre às 22h e às 8h. Animais de estimação devem ser conduzidos com coleira e deve-se usar sempre o elevador de serviço. O elevador de serviço serve para mudanças dos moradores, entre às 8h e às 18h, com intervalo do almoço entre às 12h e às 14h.",
        "A área de lazer dos idosos e das crianças funcianam das 8h às 20h.",
        "O portao tem porteiro 24 horas por dia, basta apenas se identificar.",
        "O contato do síndico, Sr. Fulano de Lima, é (81) 4321-4321.",
        "Para abertura de chamados, é necessario enviar uma mensagem pro número official do predio, através do whatsapp, que é o (81) 1234-1234",
        "Acesse a plataforma de e-learning aqui: https://www.saopaulo.sp.gov.br/spnoticias/ultimas-noticias/usp-oferece-mais-de-4-mil-vagas-em-cursos-gratuitos-para-publico-com-mais-de-60-anos/#:~:text=v%C3%ADtimas%20do%20RS-,USP%20oferece%20mais%20de%204%20mil%20vagas%20em%20cursos%20gratuitos,com%20mais%20de%2060%20anos&text=O%20programa%20USP%2060+%20completa,.usp.br/usp60. Descubra cursos gratuitos para 60+ aqui: https://geridades.com.br/2023/01/18/confira-5-cursos-gratuitos-para-idosos/. Curso gratuito de informática para 60+ aqui: https://www.uniceub.br/noticias/noticias-inclusao-digital-ceub-oferece-curso-de-informatica-gratuito-para-publico-idoso. Nesta plataforma, você encontra cursos gratuitos sobre diversos temas, como informática, idiomas, saúde e bem-estar. Aproveite esta oportunidade para aprender coisas novas e se conectar com outros moradores!",
    ]
}

df = pd.DataFrame(data)

In [97]:
# Função para gerar embeddings
def generate_embeddings(title, content):
  return genai.embed_content(
      model=model_embedding_str,
      content=content,
      title=title,
      task_type="RETRIEVAL_DOCUMENT"
  )["embedding"]

In [98]:
# Gera embeddings para o DataFrame do condomínio
df["embedding"] = df.apply(lambda row: generate_embeddings(row["title"], row["content"]), axis=1)

In [99]:
# Função para buscar a informação mais similar à pergunta do usuário
def search_similarity(query, df, model):
  query_embedding = genai.embed_content(
      model=model,
      content=query,
      task_type="RETRIEVAL_QUERY"
  )["embedding"]
  scalar_product = np.dot(np.stack(df["embedding"]), query_embedding)
  index = np.argmax(scalar_product)
  return df.iloc[index]["content"]

In [126]:
# ... (código anterior, incluindo imports, funções e criação do DataFrame)

# Inicia o chat
chat = model_generative.start_chat(history=[])
print("Olá! Sou o AntonioAssist, o seu chatbot do Condomínio Antônio Machado. 🏢 \n Como posso te ajudar? 💡\n Se quiser saber sobre notícias, digite 1 \n Se quiser saber sobre previsão do tempo, digite 2 \n🥳Você pode digitar 'FIM', em qualquer momento, para encerrar o atendimento🥳")

query = input('\nMorador: ')

while True:
  # -- VERIFICAÇÃO DE NÚMEROS -- #
  if query.strip().isdigit():
    opcao = int(query.strip())

    if opcao == 1:  # Notícias
      tópico = None
      tópico_input = input("Chatbot: Sobre qual tópico você gostaria de ver notícias?🗞️ ")
      if tópico_input:
        tópico = tópico_input.strip()
      noticias = buscar_noticias(tópico)
      if noticias:
        print("Chatbot: Aqui estão algumas notícias recentes:\n")
        for noticia in noticias:
          print(f"- {noticia['title']}\n{noticia['url']}\n")
      else:
        print("Chatbot: Desculpe, não consegui encontrar notícias no momento. Tente novamente mais tarde.")
      query = input('\nMorador: Que tal me fazer alguma outra pergunta? \nSe vc não tiver mais perguntas você pode digitar "FIM" para encerrar o atendimento ') # Leitura da entrada AQUI

    elif opcao == 2:  # Previsão do Tempo
      cidade = input("Chatbot: Para qual cidade você gostaria de saber a previsão do tempo?🌡️ ")
      previsao = buscar_previsao_tempo(cidade)
      if previsao:
        print("Chatbot:", previsao)
      else:
        print("Chatbot: Desculpe, não consegui obter a previsão do tempo para essa cidade.")
      query = input('\nMorador: Que tal me fazer alguma outra pergunta? \nSe vc não tiver mais perguntas você pode digitar "FIM" para encerrar o atendimento ') # Leitura da entrada AQUI

    else:
      print("Chatbot: Opção inválida. Por favor, digite 1 para notícias, 2 para previsão do tempo ou 'FIM' para encerrar.")
      query = input('\nMorador: Que tal me fazer alguma outra pergunta? \nSe vc não tiver mais perguntas você pode digitar "FIM" para encerrar o atendimento ') # Leitura da entrada AQUI

  # -- PERGUNTAS SOBRE O CONDOMÍNIO E OUTROS TEMAS -- #
  else:
    context = search_similarity(query, df, model_embedding_str)

    prompt = f"""Você é um chatbot que auxilia moradores do condomínio Antonio Machado, localizado no Recife - Pernambuco, fornecendo informações úteis sobre o condomínio, aprendizado e diversos temas do dia a dia, incluindo receitas, notícias, hobbies, viagens e etc..
    Responda à pergunta do morador de forma natural, clara e concisa e engajadora usando o contexto fornecido.
    Você deve auxiliar o morador com respostas relacionadas a aprendizado e sobre e-learning para qualquer idade, especialmente para os idosos.
    Se não souber a resposta, diga que não tem essa informação e sugira que o morador entre em contato com o síndico, dê o telefone de contato do mesmo e seu nome.
    Pergunta: {query}
    Contexto: {context}"""

    response = chat.send_message(prompt)
    print("Chatbot:", response.text, '\n')
    query = input('\nMorador: Que tal me fazer alguma outra pergunta? \nSe vc não tiver mais perguntas você pode digitar "FIM" para encerrar o atendimento ') # Leitura da entrada AQUI

  # -- VERIFICAÇÃO DE FIM -- #
  if query.upper() == "FIM": # Verifica se o usuário quer encerrar
    break # Sai do loop

print("Chatbot: Espero ter ajudado! Tenha um ótimo dia.")

Olá! Sou o AntonioAssist, o seu chatbot do Condomínio Antônio Machado. 🏢 
 Como posso te ajudar? 💡
 Se quiser saber sobre notícias, digite 1 
 Se quiser saber sobre previsão do tempo, digite 2 
🥳Você pode digitar 'FIM', em qualquer momento, para encerrar o atendimento🥳

Morador: 1
Chatbot: Sobre qual tópico você gostaria de ver notícias?🗞️ guerra!
Chatbot: Desculpe, não consegui encontrar notícias no momento. Tente novamente mais tarde.

Morador: Que tal me fazer alguma outra pergunta? 
Se vc não tiver mais perguntas você pode digitar "FIM" para encerrar o atendimento 2
Chatbot: Para qual cidade você gostaria de saber a previsão do tempo?🌡️ Frankfurt
Chatbot: A previsão do tempo para Frankfurt é: céu limpo, com temperatura de 19.6°C.

Morador: Que tal me fazer alguma outra pergunta? 
Se vc não tiver mais perguntas você pode digitar "FIM" para encerrar o atendimento Estou cansando, mas preciso estudar.
Chatbot: Olá, entendo que você esteja se sentindo cansado, mas saiba que estudar é es